In [1]:
import sys
sys.path.append("../")

# Single Qubit Interference

In this notebook we show a basic example of constructing sklearn transformers from gates and illustrate a use case to evaluate single qubit interference.

In [2]:
import numpy as np
import plotly.graph_objects as go
from sklearn.pipeline import make_pipeline

from skq.encoders import GateTransformer
from skq.gates import HGate, PhaseGate

In [3]:
H = GateTransformer(HGate())
P = GateTransformer(PhaseGate(np.pi/2))

These now work like any other sklearn transformer and can be used in pipelines.

However there are some constraints on the input:
- Each element must be a complex state vector with 2 elements (i.e. a qubit)
- Each element must be normalized (i.e. sum of squares of the elements must be 1)

The transformer checks these constraints.

Each `GateTransformer` uses a Unitary matrix to transform the input state(s). 

In [4]:
H

GateTransformer(gate=HGate([[ 0.70710678+0.j,  0.70710678+0.j],
       [ 0.70710678+0.j, -0.70710678+0.j]]))

In [5]:
P

GateTransformer(gate=PhaseGate([[1.000000e+00+0.j, 0.000000e+00+0.j],
           [0.000000e+00+0.j, 6.123234e-17+1.j]]))

From the transformers we can create simple singe qubit interference circuit using the class scikit-learn `Pipeline`.

In [6]:
pipe = make_pipeline(H, P, H)
pipe

Pipeline(steps=[('gatetransformer-1',
                 GateTransformer(gate=HGate([[ 0.70710678+0.j,  0.70710678+0.j],
       [ 0.70710678+0.j, -0.70710678+0.j]]))),
                ('gatetransformer-2',
                 GateTransformer(gate=PhaseGate([[1.000000e+00+0.j, 0.000000e+00+0.j],
           [0.000000e+00+0.j, 6.123234e-17+1.j]]))),
                ('gatetransformer-3',
                 GateTransformer(gate=HGate([[ 0.70710678+0.j,  0.70710678+0.j],
       [ 0.70710678+0.j, -0.70710678+0.j]])))])

This pipeline can handle multiple qubit state inputs.

In [7]:
pipe.transform(np.array([[1, 0], # |0>
                         [0, 1], # |1>
                         [1 / np.sqrt(2), 1 / np.sqrt(2)], # |+>
                         [1 / np.sqrt(2), -1 / np.sqrt(2)] # |->
                         ]))

array([[ 5.00000000e-01+0.5j       ,  5.00000000e-01-0.5j       ],
       [ 5.00000000e-01-0.5j       ,  5.00000000e-01+0.5j       ],
       [ 7.07106781e-01+0.j        ,  7.07106781e-01+0.j        ],
       [ 4.32978028e-17+0.70710678j, -4.32978028e-17-0.70710678j]])

Here we will illustrate the interference of a single qubit using a phase gate. 

By evaluating different phase gates we can evaluate and visualize the interference pattern of the qubit.

In [8]:
states = {}

for phi in np.linspace(0, 4*np.pi, 200):
    P = GateTransformer(PhaseGate(phi))
    pipe = make_pipeline(H, P, H)   
    # Input is |0>
    states[float(phi)] = pipe.transform(np.array([[1, 0]]))

In [9]:
states[0.]

array([[1.+0.j, 0.+0.j]])

In [10]:
fig = go.Figure()

xs = list(states.keys())
# |0> real part
ys0 = [s[0][0].real for s in states.values()]
# |1> real part
ys1 = [s[0][1].real for s in states.values()]
# |0> imaginary part
ys2 = [s[0][0].imag for s in states.values()]
# |1> imaginary part
ys3 = [s[0][1].imag for s in states.values()]

fig.add_trace(go.Scatter(x=xs, y=ys0, mode='lines+markers', name='|0> real part'))
fig.add_trace(go.Scatter(x=xs, y=ys1, mode='lines+markers', name='|1> real part'))
fig.add_trace(go.Scatter(x=xs, y=ys2, mode='lines+markers', name='|0> Imaginary part'))
fig.add_trace(go.Scatter(x=xs, y=ys3, mode='lines+markers', name='|1> Imaginary part'))

fig.update_layout(
    title="Single Qubit Interference over |0>",
    xaxis = dict(
        tickmode = 'array',
        tickvals = [0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi, 5*np.pi/2, 3*np.pi, 7*np.pi/2, 4*np.pi],
        ticktext = ['0', 'π/2', 'π', '3π/2', '2π', '5π/2', '3π', '7π/2', '4π']
    ),
    legend_title="Components of |ψ>"
)
fig.update_yaxes(title_text='|ψ>')
fig.update_xaxes(title_text='Phase (phi) in radians')

fig.show()


We see that the state oscillates every $2\pi$ radians, and the interference pattern is the result of the superposition of the states. 